In [1]:
#@title Sequential Version
# python3.6

import random

from paho.mqtt import client as mqtt_client
import json
import time
import threading


broker = 'localhost'
port = 1883
topic = "shared_resource/controller/#"
# Generate a Client ID with the subscribe prefix.
client_id = f'subscribe-{random.randint(0, 100)}'
publish_topic = 'colab/response'
publisher_id = f'publish-{random.randint(0, 100)}'
WINDOW_TIME = 30.0
# username = 'emqx'
# password = 'public'
fila_lock = threading.Lock()

fila = []

def connect_mqtt(id) -> mqtt_client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(id)
    # client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

def _publish(pub_topic, client, msg):
  time.sleep(1)
  print(pub_topic, msg)
  result = client.publish(pub_topic, msg)
  status = result[0]
  print(result)
  if status == 0:
      pass
  else:
      # self._set_message_received(json.dumps([f"Failed to send message to topic {self.topic}"]))
      print(f"Failed to send message to topic {pub_topic}")
  client.disconnect()  # disconnect gracefully
  client.loop_stop()  # stops network loop


def mount_message(msg, ID, _return):

  msg = {
      "id": ID,
      "return" : _return,
      "msg" : msg
  }
  print(msg)
  return json.dumps(msg)

def remove_to_queue():
  print(fila)
  fila.pop(0)
  print(fila)
  if(len(fila) > 0):
    id = fila[0]
    curr_client = connect_mqtt(publisher_id)
    msg = mount_message(f"Voce tem acesso a fila! Faca o envio das suas instrucoes.", id, "READY")
    _publish(publish_topic + f"/{id}", curr_client, msg)




#it should lock access to queue, to avoid race condition
def manager_queue(payload):
    # global fila
    # with fila_lock:
    for param in payload['params']:
      print(param)
      print("manager", fila)

      if(param == "GET_QUEUE"):
        curr_client = connect_mqtt(publisher_id)
        if(len(fila) == 0 or str(fila[0]) == str(payload['id']) ):
          msg = mount_message(f"Voce tem acesso a fila! Faca o envio das suas instrucoes.", payload['id'], "READY")
        else:
          msg = mount_message(f"Aguarde sua vez! Atualmente tem {len(fila)} pessoa(s) na fila.", payload['id'], "WAIT")
        _publish(publish_topic + f"/{payload['id']}", curr_client, msg)
        if(str(payload['id']) not in fila):
          fila.append(str(payload['id']))

      elif(param == "SEND_INSTRUCTIONS"):

          if len(fila) == 0 or str(fila[0]) != str(payload['id']):
                msg = mount_message(f"Você não está no início da fila! Atualmente tem {len(fila)} pessoa(s) na fila.", payload['id'], "WAIT")
                _publish(publish_topic + f"/{payload['id']}", curr_client, msg)
                return


          client = connect_mqtt(publisher_id)
          client.loop_start()

          instructions = payload['instructions']
          client.publish("trafficlight/id", f"{payload['id']}")
          # start_time = time.time()
          for turn in instructions:
              for instruction in turn:
                  result = client.publish(instruction['topic'], f"{instruction['operation']}")
              time.sleep(1)


          msg = mount_message(f"Sua execução terminou,.", payload['id'], "END_EXECUTION")
          _publish(publish_topic + f"/{payload['id']}", client, msg)
          remove_to_queue()







def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
        _return = json.loads(msg.payload.decode())
        manager_queue(_return)

    client.subscribe(topic)
    client.on_message = on_message

def recover_queue():
  input_file = open("backup_queue.txt",'r')
  global fila
  new_queue = input_file.readline().split(',')
  print(new_queue)
  teste = lambda x : int(x)
  fila = list(map(teste, new_queue))
  fila = new_queue
  print(fila)
  id = fila[0]
  curr_client = connect_mqtt(publisher_id)
  msg = mount_message(f"Voce tem acesso a fila! Faca o envio das suas instrucoes.", id, "READY")
  _publish(publish_topic + f"/{id}", curr_client, msg)



def run(recover = False):
    # try:
    if(recover):  recover_queue()
    client = connect_mqtt(client_id)
    subscribe(client)
    client.loop_forever()
    # except:
    #   print("error")
    #   if(len(fila) > 0):
    #     arq = open("backup_queue.txt","w")
    #     arq.write(",".join(fila))


if __name__ == '__main__':
    run(recover=False)


Connected to MQTT Broker!


KeyboardInterrupt: 

In [2]:
# Attempt Parallel access

import random

from paho.mqtt import client as mqtt_client
import json
import time
import threading
import concurrent.futures


broker = 'localhost'
port = 1883
topic = "shared_resource/controller/#"
# Generate a Client ID with the subscribe prefix.
client_id = f'subscribe-{random.randint(0, 100)}'
publish_topic = 'colab/response'
publisher_id = f'publish-{random.randint(0, 100)}'
WINDOW_TIME = 35.0
# username = 'emqx'
# password = 'public'
fila_lock = threading.Lock()

fila = []

def connect_mqtt(id) -> mqtt_client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(id)
    # client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

def _publish(pub_topic, client, msg):
  time.sleep(1)
  print(pub_topic, msg)
  result = client.publish(pub_topic, msg)
  status = result[0]
  print(result)
  if status == 0:
      pass
  else:
      # self._set_message_received(json.dumps([f"Failed to send message to topic {self.topic}"]))
      print(f"Failed to send message to topic {pub_topic}")
  client.disconnect()  # disconnect gracefully
  client.loop_stop()  # stops network loop


def clear_resource(client):

    try:
        client.publish("trafficlight/id", f"_")

        client.publish("trafficlight/green", 0)
        client.publish("trafficlight/yellow", 0)
        client.publish("trafficlight/red", 0)

    except Exception as e:
        print(f"Exception: {e}")


def mount_message(msg, ID, _return):

  msg = {
      "id": ID,
      "return" : _return,
      "msg" : msg
  }
  print(msg)
  return json.dumps(msg)

#locks write access to queue
def remove_to_queue():
  with fila_lock:
    print(fila)
    fila.pop(0)
    print(fila)
    if(len(fila) > 0):
      id = fila[0]
      curr_client = connect_mqtt(publisher_id)
      msg = mount_message(f"Voce tem acesso a fila! Faca o envio das suas instrucoes.", id, "READY")
      _publish(publish_topic + f"/{id}", curr_client, msg)


def send_inst_to_resource(payload):
  client = connect_mqtt(publisher_id)
  client.loop_start()

  instructions = payload['instructions']
  client.publish("trafficlight/id", f"{payload['id']}")

  start_time = time.time()

  _send_instructions = True
  def expiration_callback():
      msg = mount_message(f"Tempo de execução máximo atingido ({WINDOW_TIME} segundos).", payload['id'], "TIMEOUT")
      _publish(publish_topic + f"/{payload['id']}", client, msg)
      nonlocal _send_instructions
      _send_instructions = False
      remove_to_queue()
      return

  timer = threading.Timer(WINDOW_TIME, expiration_callback)
  timer.start()

  for turn in instructions:
    if(not _send_instructions): break

    for instruction in turn:
        result = client.publish(instruction['topic'], f"{instruction['operation']}")
    time.sleep(1)

  # Stop the timer as the execution finished within the time window
  timer.cancel()
  clear_resource(client)

  if(_send_instructions):
    msg = mount_message(f"Sua execução terminou.", payload['id'], "END_EXECUTION")
    _publish(publish_topic + f"/{payload['id']}", client, msg)
    remove_to_queue()

#it should lock access to queue, to avoid race condition
def manager_queue(payload, mode = "Run"):
    if mode == "Testbench":
      curr_client = connect_mqtt(publisher_id)
      msg = mount_message(f"Aguarde sua vez! Atualmente tem {len(fila)} pessoa(s) na fila.", payload['id'], "WAIT")
      _publish(publish_topic + f"/{payload['id']}", curr_client, msg)
      return


    for param in payload['params']:
      print(param)
      print("manager", fila)

      if(param == "GET_QUEUE"):
        curr_client = connect_mqtt(publisher_id)


        #avoid read queue when other request is writing
        with fila_lock:
          if(len(fila) == 0 or str(fila[0]) == str(payload['id']) ):
            msg = mount_message(f"Voce tem acesso a fila! Faca o envio das suas instrucoes.", payload['id'], "READY")
          else:
            msg = mount_message(f"Aguarde sua vez! Atualmente tem {len(fila)} pessoa(s) na fila.", payload['id'], "WAIT")
          _publish(publish_topic + f"/{payload['id']}", curr_client, msg)
          if(str(payload['id']) not in fila):
            fila.append(str(payload['id']))


      elif(param == "SEND_INSTRUCTIONS"):

          with fila_lock:
            if len(fila) == 0 or str(fila[0]) != str(payload['id']):
              msg = mount_message(f"Você não está no início da fila! Atualmente tem {len(fila)} pessoa(s) na fila.", payload['id'], "WAIT")
              _publish(publish_topic + f"/{payload['id']}", curr_client, msg)
              return


          inst_thread = threading.Thread(target=send_inst_to_resource, args=(payload,))
          inst_thread.start()
          return

def subscribe(client: mqtt_client, mode: str):
    def on_message(client, userdata, msg):
        print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
        _return = json.loads(msg.payload.decode())
        manager_queue(_return, mode)

    client.subscribe(topic)
    client.on_message = on_message

def recover_queue():
  input_file = open("backup_queue.txt",'r')
  global fila
  new_queue = input_file.readline().split(',')
  print(new_queue)
  teste = lambda x : int(x)
  fila = list(map(teste, new_queue))
  fila = new_queue
  print(fila)
  id = fila[0]
  curr_client = connect_mqtt(publisher_id)
  msg = mount_message(f"Voce tem acesso a fila! Faca o envio das suas instrucoes.", id, "READY")
  _publish(publish_topic + f"/{id}", curr_client, msg)



def run(recover = False, mode = "Run"):
    # try:
    if(recover):  recover_queue()
    client = connect_mqtt(client_id)
    subscribe(client, mode)
    client.loop_forever()
    # except:
    #   print("error")
    #   if(len(fila) > 0):
    #     arq = open("backup_queue.txt","w")
    #     arq.write(",".join(fila))


if __name__ == '__main__':
    run(recover=False, mode = "Run")


Connected to MQTT Broker!
Received `{"id": 1, "instructions": [], "generate_time": 0, "send_time": 0, "session_start_time": "9_40_57", "params": ["GET_QUEUE"]}` from `shared_resource/controller/1` topic
GET_QUEUE
manager []
{'id': 1, 'return': 'READY', 'msg': 'Voce tem acesso a fila! Faca o envio das suas instrucoes.'}
colab/response/1 {"id": 1, "return": "READY", "msg": "Voce tem acesso a fila! Faca o envio das suas instrucoes."}
(0, 1)
Received `{"id": "1", "instructions": [[{"topic": "trafficlight/green", "operation": "1"}, {"topic": "trafficlight/yellow", "operation": "1"}, {"topic": "trafficlight/red", "operation": "1"}], [{"topic": "trafficlight/green", "operation": "0"}, {"topic": "trafficlight/yellow", "operation": "0"}, {"topic": "trafficlight/red", "operation": "0"}], [{"topic": "trafficlight/green", "operation": "1"}, {"topic": "trafficlight/yellow", "operation": "1"}, {"topic": "trafficlight/red", "operation": "1"}], [{"topic": "trafficlight/green", "operation": "0"}, {"top

KeyboardInterrupt: 